In [21]:
#predefines and imports
import numpy as np
import pandas as pd

In [99]:
def init_probability_stw():
    df = pd.DataFrame(columns = ['Weather', 'Temperature', 'Season'])
    SUMMER , WINTER = 'summer', 'winter'
    HOT, COLD = 'hot','cold'
    SUN, RAIN = 'sun', 'rain'
    grid = np.array(np.meshgrid([SUMMER,WINTER],[HOT,COLD],[SUN,RAIN])).T.reshape(-1,3)
    # print(grid)
    for row in grid:
        df = df.append({'Season':row[0], 'Temperature':row[1], 'Weather':row[2]}, ignore_index=True)
    probability = np.array([0.3,0.1,0.1,0.15,0.05,0.05,0.05,0.2])
    df['P'] = probability
    return df
def init2():    
    season = ['summer', 'winter']
    temperature = ['hot','cold']
    weather = ['sun','rain']
    grid =  np.array(np.meshgrid(season,temperature,weather)).T.reshape(-1,3)
    for col in range(grid.shape[1]-1,-1,-1):
        grid = grid[grid[:, col].argsort()]
    grid = pd.DataFrame(grid,columns = ['Season','Temperature','Weather'])
    index = pd.MultiIndex.from_frame(grid)

    probability =np.array([0.05,0.1,0.05,0.3,0.2,0.15,0.05,0.1])
    df = pd.Series(data = probability ,index = index, dtype = np.float64)
    return df
def check_is_prob_dist(df):
    p = df.P
    for prob in p:
        if prob < 0: return False
    if p.sum() != 1: return False
    return True

# data = init_probability_stw()
# print(check_is_prob_dist(data))
# data
data = init2()
display(data)

Season  Temperature  Weather
summer  cold         rain       0.05
                     sun        0.10
        hot          rain       0.05
                     sun        0.30
winter  cold         rain       0.20
                     sun        0.15
        hot          rain       0.05
                     sun        0.10
dtype: float64

In [322]:
def cond_prob(data, queues, evidences):
    '''
    queue: list of name of queue att
    evidences: list of (name, value) of evidences
    '''
    #NOTE: example: P ( q1,q2 | e1,e2,e3)
    # values = tuple([e[1] for e in evidences]) if len(evidences) > 1 else evidences[0]
    selected = select(data,evidences)
    result, _ = marginal_prob(selected,queues) # remove hidden by grouping and normalize

    que = ', '.join(queues)
    evi = ', '.join([str(x[0]) + '=' + str(x[1]) for x in evidences])

    return result , f'Cond Prob: ({que}) given ({evi})\n'
def cond_dist(data, queues, evidences):
    #evidences here are just names
    temp , _ = marginal_prob(data,queues + evidences)
    evi_values = [x if isinstance(x,tuple) else (x,) for x in marginal_prob(data, evidences)[0].index]

    for values in evi_values:

        prob,_ = cond_prob(data,queues, list(zip(evidences,values)))
        for idx in prob.index:
            res = prob.loc[idx]
            temp.loc[idx].loc[values] = res
    
    return sort_series_by(temp,order = evidences + queues), f'Cond Prob: ({queues}) given ({evidences})\n'
def select(data,evidences):
    selected = data
    for e in evidences:
        selected = selected.iloc[selected.index.get_level_values(e[0]) == e[1]]
    return selected

def marginal_prob(data, attributes):
    s = data.groupby(attributes).sum()
    return normalize(s), f'Marginal {attributes}\n'

def sort_series_by(data, order):
    temp =  data
    temp = temp.reorder_levels(order)
    return temp.sort_index()
def normalize(data):
    return data/data.sum()

def joint_chain(prob, cond):
    temp = cond
    for pidx in prob.index:
        for cidex in cond.index:
            pi = pidx if isinstance(pidx,tuple) else (pidx,)
            ci = cidex if isinstance(cidex,tuple) else (cidex,)
            isIn = all([p in ci for p in pi])
            if isIn: 
                temp[cidex] = temp[cidex] * prob[pidx]
    unknown_name = [x for x in cond.index.names if (not x in prob.index.names)]
    return temp, f'Joint of {cond.index.names} knowing ({unknown_name} given {prob.index.name}) and {prob.index.name}\n'
def bayes(data, queues, evidences):
    pass
def independence(data,first, second, given):
    #! does not work with cond independence or multi var independence
    p_a,m1 = marginal_prob(data,first) if given is None else cond_dist(data,first,given)
    p_b,m2 = marginal_prob(data,second) if given is None else cond_dist(data,second,given)
    p_ab,m3 = marginal_prob(data,first + second) if given is None else cond_dist(data, first + second, given)
    print(m1,p_a)
    print(m2,p_b)
    print(m3,p_ab)

    x = p_ab.copy()
    leng = 0 if given is None else len(given)
    lena = len(first) + leng
    lenb = len(second) + leng
    for idx in x.index:
        print(idx)
        print('idxa: ',idx[:lena],'pa: ',p_a.loc[idx[:lena]])
        print('idxb: ',idx[:lenb],'pb: ',p_b.loc[idx[:lenb]])
        a = p_a.loc[idx[:lena]]
        b = p_b.loc[idx[lenb:]]
        x.loc[idx] = a*b
    print(x, 'equal: ', (x==p_ab).all())

independence(data,['Season'],['Temperature'],None)

Marginal ['Season']
 Season
summer    0.5
winter    0.5
dtype: float64
Marginal ['Temperature']
 Temperature
cold    0.5
hot     0.5
dtype: float64
Marginal ['Season', 'Temperature']
 Season  Temperature
summer  cold           0.15
        hot            0.35
winter  cold           0.35
        hot            0.15
dtype: float64
('summer', 'cold')
idxa:  ('summer',) pa:  0.5


KeyError: 'summer'

In [314]:
def test_cond_prob(data):
    t1,m1  = cond_prob(data,['Weather'],[('Season','summer'),('Temperature','hot')])
    print(m1,t1)
    t2,m2  = cond_prob(data,['Weather','Season'],[('Temperature','hot')])
    print(t2,m2)
def test_chain(data):
    cond,mes1 = cond_dist(data,['Weather'],[('Season')])
    prob,mes2 = marginal_prob(data,['Season'])
    print(mes1,cond)
    print(mes2,prob)
    chain,mes3 = joint_chain(prob,cond)
    print(mes3,chain)
def test_cond_dist(data):
    ret, mes = cond_dist(data, ['Weather'], ['Season'])
    print(mes)
    display(ret)
# test_chain(data)